In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("sheep\2013-12-03")

create_database('blp_dataset')
create_table('products_tmp_2')

In [ ]:
original_images = []

folder = Path(f"{MAIN_FOLDER}\market\item")

for file in os.listdir(folder):
    page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
    if page_content is not None:
        ## GET DESCRIPTION
        raw_description = page_content.find("td", attrs={"style": "background-color: #F8F8F8;"})
        description = get_content(raw_description)

        ## GET TITLE
        raw_title = page_content.find("h4")
        title = get_content(raw_title)

        ## GET SELLER
        raw_seller = page_content.find("a", attrs={"title": "Send a message"})
        seller = get_content(raw_seller)

        if description != "" or title != "":
            # Insert product into database
            query = "INSERT INTO products_tmp_2 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
            values = (title, description, "Sheep", seller_name, "t")

            cursor.execute(query, values)
            conn.commit()

            product_id = cursor.lastrowid

            ## GET IMAGE
            raw_image = page_content.find("img")
            
            if raw_image is not None:
                try:
                    image = raw_image["src"]
                except IndexError:
                    print(f"Image {product_id} has no src")
                    update_image(product_id, "no_image")
                
                if image is not "":
                    try:
                        data = "" + image.replace("data:image/jpeg;base64,", "")
                        imgdata = base64.b64decode(data)

                        img_filename = Path(f"D:\images\p\{product_id}.jpg")

                        with open(img_filename, 'wb') as f:
                            f.write(imgdata)
                            f.close
                    except:
                        print(f"Image {product_id} has error. Except")
                        update_image(product_id, "no_image")
            else:
                    print(f"Image {product_id} has no image")
                    update_image(product_id, "no_image")